In [1]:
%load_ext lab_black
%load_ext autotime
import pandas as pd
import numpy as np

time: 291 ms (started: 2022-09-11 21:26:02 -07:00)


This notebook relies heavily on various other notebooks that know what they are doing from the biological side of things to a far greater extent than me:

* [openTSNE notebook](https://github.com/pavlin-policar/openTSNE/blob/30b07f1b60129fc74bb937eb4632c6d424f49c4d/examples/prepare_macosko_2015.ipynb) -- this one in particular
* [BerensLab notebook](https://github.com/berenslab/umi-normalization/blob/main/07_compute_retina.ipynb)
* [hemberg-lab shell script](https://github.com/hemberg-lab/scRNA.seq.datasets/blob/8fd6ec9b85917e3f4a0c097006083e9c9e62cde8/bash/macosko.sh)

**Warning**: we are dealing with a big file here. While it's only 51 MB to download, it's 2.3 GB when unzipped and while it has a mere 25000 rows (why, that's not even *half* an MNIST!), it has nearly 50000 columns. So reading this in takes a *long* time. Make sure you have the time (and RAM) to do this: The final dataframe takes up nearly 10 GB of RAM and there is a spike of up to nearly 12 GB just before it finishes.

## Reading in the data the hard way

To help with handling the memory requirements, we are only going to use the columns which are listed in this file of cluster identities:

In [6]:
cluster_ids = pd.read_csv(
    "https://scrnaseq-public-datasets.s3.amazonaws.com/manual-data/macosko/retina_clusteridentities.txt",
    header=None,
    index_col=0,
    sep="\t",
).squeeze(1)

time: 1.64 s (started: 2022-09-11 21:26:57 -07:00)


In [3]:
cluster_ids.index

Index(['r1_GGCCGCAGTCCG', 'r1_CTTGTGCGGGAA', 'r1_GCGCAACTGCTC',
       'r1_GATTGGGAGGCA', 'r1_GTGCCGCCTCTC', 'r1_CCTGTGACACAC',
       'r1_AATCTCGTTAAT', 'r1_GATTTCCTCTGA', 'r1_GAAGGCTGGAAC',
       'r1_TTCCGGCTGTAC',
       ...
       'p1_TGTACACCTGGG', 'p1_GAGGGGCTCTAA', 'p1_AGCCAAGGCTCG',
       'p1_TGAGTCGTCTTA', 'p1_CGAATACGTGTC', 'p1_TCAAAAGCCGGG',
       'p1_ATTAAGTTCCAA', 'p1_CTGTCTGAGACC', 'p1_TAACGCGCTCCT',
       'p1_ATTCTTGTTCTT'],
      dtype='object', name=0, length=44808)

time: 9.54 ms (started: 2022-09-10 21:54:03 -07:00)


There are 44808 cells, which is a lot of columns but we can still avoid reading nearly 5000 columns worth of data. Also, the largest count in the columns we care about is only `671` (it's `679` for the entire dataset but we won't use that column), which means we can use `np.int16` for the `dtype` of the dataframe, rather than the typical `np.int64`. This should save a factor of 4 in each column's storage (that can be checked by looking at the output of `data.memory_usage(index=False, deep=True)` if you so wish).

The downside is that we can't read the string-based index in at the same time (we'll do that by reading in separately), but doing things this way results in the RAM usage being below 3GB. Also, we need to read in `cluster_ids` anyway for our `target` (label) data, so we are just making it work for us early.

In [4]:
data = pd.read_csv(
    "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE63472&format=file&file=GSE63472%5FP14Retina%5Fmerged%5Fdigital%5Fexpression%2Etxt%2Egz",
    sep="\t",
    compression="gzip",
    header=0,
    chunksize=400,
    dtype=np.int16,
    usecols=lambda x: x in cluster_ids.index,
)

time: 1min 12s (started: 2022-09-10 20:18:46 -07:00)


Still here? Yeah, well that's because of the use of `chunksize` nothing really happens until you try to create the complete dataframe, which we will do below. Good luck everyone...

In [5]:
data = pd.concat(data)

time: 10min 21s (started: 2022-09-10 20:20:04 -07:00)


This can take quite a while (e.g. over 10 minutes for me). If your kernel didn't survive that you may have to fiddle with the `chunksize`. Once that works we need the index column which is column `0`:

In [7]:
data_index = pd.read_csv(
    "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE63472&format=file&file=GSE63472%5FP14Retina%5Fmerged%5Fdigital%5Fexpression%2Etxt%2Egz",
    sep="\t",
    compression="gzip",
    header=0,
    index_col=0,
    chunksize=400,
    usecols=[0],
)
data_index = pd.concat(data_index)

time: 1min 57s (started: 2022-09-10 20:34:52 -07:00)


And now add the index to `data`:

In [8]:
data.set_index(data_index.index, inplace=True)

time: 1.04 ms (started: 2022-09-10 20:36:56 -07:00)


This process is a bit fiddly and slow (although not a lot slower than the more straightforward approach), but it did mean my kernel did not exceed ~2.6GB of RAM usage.

## Reading in the data the easy (but RAM-consuming) way

If you have lots of RAM and all of the above seems a real pain, then you can just do the following, but I have commented it out, because I would hate to run this by accident:

In [9]:
# NOT a good idea unless you have lots of RAM
# data = pd.read_csv(
#     "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE63472&format=file&file=GSE63472%5FP14Retina%5Fmerged%5Fdigital%5Fexpression%2Etxt%2Egz",
#     sep="\t",
#     compression="gzip",
#     header=0,
#     index_col=0,
# )
# remove the columns we don't need
# data = data.loc[:, data.columns.isin(cluster_ids.index)]

time: 697 µs (started: 2022-09-10 20:01:46 -07:00)


## Save the intermediate data

However you do it, just to be on the safe side you might want to consider pickling that file to disk before moving on (gzipping will reduce the size of the `np.int64` version of the output file from 10 GB to 70 MB but can take a few minutes to do its work -- the `np.int16` version is around 50 MB compressed):

In [42]:
import pickle
import drnb.io as nbio
import gzip

# uncomment this if you just want the pickle
# with open(
#     nbio.DATA_ROOT / "data" / "GSE63472_P14Retina_merged_digital_expression.pkl", "wb"
# ) as f:
#     pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

# uncomment this if you want the pickle gzipped too
# with gzip.open(
#     nbio.DATA_ROOT / "data" / "GSE63472_P14Retina_merged_digital_expression.pkl.gz",
#     "wb",
# ) as f:
#     pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

time: 1.22 ms (started: 2022-09-11 21:42:22 -07:00)


## Load the intermediate data

If something goes wrong, get the data back. If you aren't using the `np.int16` version of the data, be aware that although this is several times faster than writing, you may see a memory spike (in my case, up to 16 GB) just before the data is returned:

In [41]:
# import pickle
# import drnb.io as nbio
# import gzip

# uncomment this if you wrote the pickle file
# with open(
#     nbio.DATA_ROOT / "data" / "GSE63472_P14Retina_merged_digital_expression.pkl",
#     "rb",
# ) as f:
#     data = pickle.load(f)

# uncomment this if you wrote the gzipped pickle file
# with gzip.open(
#     nbio.DATA_ROOT / "data" / "GSE63472_P14Retina_merged_digital_expression.pkl.gz",
#     "rb",
# ) as f:
#     data = pickle.load(f)

time: 836 µs (started: 2022-09-11 21:42:20 -07:00)


Also remember to load `cluster_ids` back in (near the top of this document).

In [3]:
data.shape

(24658, 44808)

time: 11.2 ms (started: 2022-09-11 21:26:50 -07:00)


## Name the clusters by cell type

Back to the clusters for a bit (this bit is taken from  [openTSNE notebook](https://github.com/pavlin-policar/openTSNE/blob/30b07f1b60129fc74bb937eb4632c6d424f49c4d/examples/prepare_macosko_2015.ipynb)).

In [7]:
cluster_ids.head()

0
r1_GGCCGCAGTCCG     2
r1_CTTGTGCGGGAA     2
r1_GCGCAACTGCTC     2
r1_GATTGGGAGGCA     2
r1_GTGCCGCCTCTC    25
Name: 1, dtype: int64

time: 5.98 ms (started: 2022-09-11 21:27:02 -07:00)


Reorder `cluster_ids` so the contents are in the same order as the columns of `data`:

In [8]:
cluster_ids = cluster_ids[data.columns.values]

time: 23.1 ms (started: 2022-09-11 21:27:05 -07:00)


In [9]:
cluster_ids.head()

0
r1_GGCCGCAGTCCG     2
r1_CTTGTGCGGGAA     2
r1_GCGCAACTGCTC     2
r1_GATTGGGAGGCA     2
r1_GTGCCGCCTCTC    25
Name: 1, dtype: int64

time: 4.69 ms (started: 2022-09-11 21:27:06 -07:00)


Same order as the dataframe:

In [10]:
data.columns.values[:5]

array(['r1_GGCCGCAGTCCG', 'r1_CTTGTGCGGGAA', 'r1_GCGCAACTGCTC',
       'r1_GATTGGGAGGCA', 'r1_GTGCCGCCTCTC'], dtype=object)

time: 4.68 ms (started: 2022-09-11 21:27:11 -07:00)


There should also not be cells with missing data:

In [11]:
cluster_ids.isna().any()

False

time: 3.64 ms (started: 2022-09-11 21:27:12 -07:00)


The cluster ids are all numeric:

In [12]:
cluster_ids.value_counts().head()

24    29400
26     2217
25     1868
34     1624
33      849
Name: 1, dtype: int64

time: 5.82 ms (started: 2022-09-11 21:27:13 -07:00)


But they do correspond to a cell type:

In [13]:
cell_types = cluster_ids.astype(object)

cell_types.loc[cell_types == 1] = "Horizontal cells"
cell_types.loc[cell_types == 2] = "Retinal ganglion cells"
cell_types.loc[cell_types.isin(range(3, 24))] = "Amacrine cells"
cell_types.loc[cell_types == 24] = "Rods"
cell_types.loc[cell_types == 25] = "Cones"
cell_types.loc[cell_types.isin(range(26, 34))] = "Bipolar cells"
cell_types.loc[cell_types == 34] = "Muller glia"
cell_types.loc[cell_types == 35] = "Astrocytes"
cell_types.loc[cell_types == 36] = "Fibroblasts"
cell_types.loc[cell_types == 37] = "Vascular endothelium"
cell_types.loc[cell_types == 38] = "Pericytes"
cell_types.loc[cell_types == 39] = "Microglia"

cell_types.value_counts().head()

Rods              29400
Bipolar cells      6285
Amacrine cells     4426
Cones              1868
Muller glia        1624
Name: 1, dtype: int64

time: 43 ms (started: 2022-09-11 21:27:21 -07:00)


## Gene selection

Requires converting `data` to sparse format. All the functions here are from the [openTSNE notebook](https://github.com/pavlin-policar/openTSNE/blob/30b07f1b60129fc74bb937eb4632c6d424f49c4d/examples/prepare_macosko_2015.ipynb) (lightly modified to only include the needed functionality).

In [14]:
import scipy.sparse as sp

counts = sp.csr_matrix(data.values)
counts

<24658x44808 sparse matrix of type '<class 'numpy.int16'>'
	with 32805477 stored elements in Compressed Sparse Row format>

time: 17.7 s (started: 2022-09-11 21:27:29 -07:00)


In [15]:
def select_genes(
    data,
    n,
    threshold=0,
    atleast=10,
    yoffset=0.02,
    xoffset=5,
    decay=1,
):
    zeroRate = 1 - np.squeeze(np.array((data > threshold).mean(axis=0)))
    A = data.multiply(data > threshold)
    A.data = np.log2(A.data)
    meanExpr = np.zeros_like(zeroRate) * np.nan
    detected = zeroRate < 1
    meanExpr[detected] = np.squeeze(np.array(A[:, detected].mean(axis=0))) / (
        1 - zeroRate[detected]
    )

    lowDetection = np.array(np.sum(data > threshold, axis=0)).squeeze() < atleast
    zeroRate[lowDetection] = np.nan
    meanExpr[lowDetection] = np.nan

    up = 10
    low = 0
    for _ in range(100):
        nonan = ~np.isnan(zeroRate)
        selected = np.zeros_like(zeroRate).astype(bool)
        selected[nonan] = (
            zeroRate[nonan] > np.exp(-decay * (meanExpr[nonan] - xoffset)) + yoffset
        )
        if np.sum(selected) == n:
            break
        if np.sum(selected) < n:
            up = xoffset
            xoffset = (xoffset + low) / 2
        else:
            low = xoffset
            xoffset = (xoffset + up) / 2
    print(f"Chosen offset: {xoffset:.2f}")

    return selected

time: 9.38 ms (started: 2022-09-11 21:27:46 -07:00)


In [16]:
gene_mask = select_genes(counts.T, n=3000)

Chosen offset: 0.19
time: 999 ms (started: 2022-09-11 21:27:50 -07:00)


## Create the final log-cpm data

Because of the very skewed nature of the count data, the final form of the data is stored as the (one plus the) log of the counts per million (CPM).

In [17]:
def calculate_cpm(x):
    """Calculate counts-per-million on data where the rows are genes."""
    normalization = np.sum(x, axis=0)
    # On sparse matrices, the sum will be 2d. We want a 1d array
    normalization = np.squeeze(np.asarray(normalization))
    # Straight up division is not an option since this will form a full dense
    # matrix if `x` is sparse. Divison can be expressed as the dot product with
    # a reciprocal diagonal matrix
    normalization = sp.diags(1 / normalization, offsets=0)
    cpm_counts = np.dot(x, normalization)

    return cpm_counts * 1e6

time: 2.55 ms (started: 2022-09-11 21:27:55 -07:00)


In [18]:
cpm_counts = calculate_cpm(counts)
cpm_counts

<24658x44808 sparse matrix of type '<class 'numpy.float64'>'
	with 32805477 stored elements in Compressed Sparse Row format>

time: 690 ms (started: 2022-09-11 21:27:55 -07:00)


In [19]:
def log_normalize(data):
    """Perform log transform log(x + 1)"""
    data = data.copy()
    data.data = np.log2(data.data + 1)
    return data

time: 2.72 ms (started: 2022-09-11 21:28:33 -07:00)


In [20]:
log_counts = log_normalize(cpm_counts)
log_counts

<24658x44808 sparse matrix of type '<class 'numpy.float64'>'
	with 32805477 stored elements in Compressed Sparse Row format>

time: 456 ms (started: 2022-09-11 21:28:33 -07:00)


Then the log cpm data is transposed, so the cells are rows and the genes are columns and only the 3000 selected genes are kept:

In [21]:
x = log_counts.T[:, gene_mask].toarray()
x.shape

(44808, 3000)

time: 668 ms (started: 2022-09-11 21:28:36 -07:00)


In [22]:
x.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

time: 2.96 ms (started: 2022-09-11 21:28:37 -07:00)


In [26]:
target = pd.DataFrame(dict(ClusterId=cluster_ids, CellType=cell_types))

,ClusterId,CellType
0,,
r1_GGCCGCAGTCCG,2,Retinal ganglion cells
r1_CTTGTGCGGGAA,2,Retinal ganglion cells
r1_GCGCAACTGCTC,2,Retinal ganglion cells
r1_GATTGGGAGGCA,2,Retinal ganglion cells
r1_GTGCCGCCTCTC,25,Cones
...,...,...
p1_TCAAAAGCCGGG,24,Rods
p1_ATTAAGTTCCAA,34,Muller glia
p1_CTGTCTGAGACC,2,Retinal ganglion cells


time: 9.71 ms (started: 2022-09-11 21:31:31 -07:00)


While we're at it, let's get rid of that `0` as the index name:

In [33]:
target.index.name = None

time: 848 µs (started: 2022-09-11 21:33:47 -07:00)


## Data Pipeline

In [34]:
from drnb.dataset import create_data_pipeline

data_pipe = create_data_pipeline(
    convert=dict(dtype="float32", layout="c"),
    data_export=["csv", "npy"],
    target_export=["csv", "pkl"],
    neighbors=dict(
        n_neighbors=[15, 50, 150],
        method="exact",
        metric=["euclidean"],
        file_types=["csv", "npy"],
    ),
    triplets=dict(
        n_triplets_per_point=5,
        seed=1337,
        file_types=["csv", "npy"],
    ),
    verbose=True,
)

[21:34:14] INFO     Requesting one extra neighbor to account for self-neighbor                       ]8;id=951968;file:///home/james/dev/drnb/src/drnb/dataset.py\dataset.py]8;;\:]8;id=980267;file:///home/james/dev/drnb/src/drnb/dataset.py#310\310]8;;\

time: 13.6 ms (started: 2022-09-11 21:34:14 -07:00)


Also we will include a palette used for plotting colors when using the cell types as a legend (once again from openTSNE):

In [35]:
target_palette = dict(
    CellType={
        "Amacrine cells": "#A5C93D",
        "Astrocytes": "#8B006B",
        "Bipolar cells": "#2000D7",
        "Cones": "#538CBA",
        "Fibroblasts": "#8B006B",
        "Horizontal cells": "#B33B19",
        "Microglia": "#8B006B",
        "Muller glia": "#8B006B",
        "Pericytes": "#8B006B",
        "Retinal ganglion cells": "#C38A1F",
        "Rods": "#538CBA",
        "Vascular endothelium": "#8B006B",
    }
)

time: 3.12 ms (started: 2022-09-11 21:34:27 -07:00)


In [36]:
data_result = data_pipe.run(
    "macosko2015", data=x, target=target, target_palette=target_palette, verbose=True
)

[21:34:29] INFO     initial data shape: (44808, 3000) target shape: (44808, 2)                        ]8;id=716603;file:///home/james/dev/drnb/src/drnb/dataset.py\dataset.py]8;;\:]8;id=714935;file:///home/james/dev/drnb/src/drnb/dataset.py#45\45]8;;\

           INFO     Removing rows with NAs                                                            ]8;id=923677;file:///home/james/dev/drnb/src/drnb/dataset.py\dataset.py]8;;\:]8;id=422656;file:///home/james/dev/drnb/src/drnb/dataset.py#85\85]8;;\

[21:34:32] INFO     data shape after filtering NAs: (44808, 3000)                                     ]8;id=224965;file:///home/james/dev/drnb/src/drnb/dataset.py\dataset.py]8;;\:]8;id=624126;file:///home/james/dev/drnb/src/drnb/dataset.py#94\94]8;;\

           INFO     Keeping all columns                                                            ]8;id=453101;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=135597;file:///home/james/dev/drnb/src/drnb/preprocess.py#62\62]8;;\

           INFO     data shape after filtering columns: (44808, 3000)                                 ]8;id=194160;file:///home/james/dev/drnb/src/drnb/dataset.py\dataset.py]8;;\:]8;id=955112;file:///home/james/dev/drnb/src/drnb/dataset.py#99\99]8;;\

...

[21:35:56] INFO     Writing csv format to triplets/macosko2015.5.1337.l2.csv                        ]8;id=53403;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=878459;file:///home/james/dev/drnb/src/drnb/io/__init__.py#207\207]8;;\

[21:35:57] INFO     Writing numpy format to triplets/macosko2015.5.1337.idx.npy                     ]8;id=126896;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=717179;file:///home/james/dev/drnb/src/drnb/io/__init__.py#230\230]8;;\

           INFO     Writing numpy format to triplets/macosko2015.5.1337.l2.npy                      ]8;id=61658;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=31204;file:///home/james/dev/drnb/src/drnb/io/__init__.py#230\230]8;;\

           INFO     Writing pipeline result for macosko2015                                           ]8;id=873522;file:///home/james/dev/drnb/src/drnb/dataset.py\dataset.py]8;;\:]8;id=606931;file:///home/james/dev/drnb/src/drnb/dataset.py#79\79]8;;\

time: 1min 27s (started: 2022-09-11 21:34:29 -07:00)


The openTSNE notebook standardizes the data, so here's a pipeline that does that too (via `scale="z"`):

In [37]:
standardize_pipe = create_data_pipeline(
    convert=dict(dtype="float32", layout="c"),
    scale="z",
    data_export=["csv", "npy"],
    target_export=["csv", "pkl"],
    neighbors=dict(
        n_neighbors=[15, 50, 150],
        method="exact",
        metric=["euclidean"],
        file_types=["csv", "npy"],
    ),
    triplets=dict(
        n_triplets_per_point=5,
        seed=1337,
        file_types=["csv", "npy"],
    ),
    verbose=True,
)

[21:36:07] INFO     Requesting one extra neighbor to account for self-neighbor                       ]8;id=212396;file:///home/james/dev/drnb/src/drnb/dataset.py\dataset.py]8;;\:]8;id=426197;file:///home/james/dev/drnb/src/drnb/dataset.py#310\310]8;;\

time: 8.39 ms (started: 2022-09-11 21:36:07 -07:00)


In [38]:
data_result = standardize_pipe.run(
    "macosko2015z", data=x, target=target, target_palette=target_palette, verbose=True
)

[21:36:22] INFO     initial data shape: (44808, 3000) target shape: (44808, 2)                        ]8;id=18557;file:///home/james/dev/drnb/src/drnb/dataset.py\dataset.py]8;;\:]8;id=430279;file:///home/james/dev/drnb/src/drnb/dataset.py#45\45]8;;\

           INFO     Removing rows with NAs                                                            ]8;id=572133;file:///home/james/dev/drnb/src/drnb/dataset.py\dataset.py]8;;\:]8;id=858768;file:///home/james/dev/drnb/src/drnb/dataset.py#85\85]8;;\

[21:36:24] INFO     data shape after filtering NAs: (44808, 3000)                                     ]8;id=973971;file:///home/james/dev/drnb/src/drnb/dataset.py\dataset.py]8;;\:]8;id=978366;file:///home/james/dev/drnb/src/drnb/dataset.py#94\94]8;;\

           INFO     Keeping all columns                                                            ]8;id=937511;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=117515;file:///home/james/dev/drnb/src/drnb/preprocess.py#62\62]8;;\

           INFO     data shape after filtering columns: (44808, 3000)                                 ]8;id=251502;file:///home/james/dev/drnb/src/drnb/dataset.py\dataset.py]8;;\:]8;id=965996;file:///home/james/dev/drnb/src/drnb/dataset.py#99\99]8;;\

...

[21:38:31] INFO     Writing csv format to triplets/macosko2015z.5.1337.l2.csv                       ]8;id=516096;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=283500;file:///home/james/dev/drnb/src/drnb/io/__init__.py#207\207]8;;\

[21:38:32] INFO     Writing numpy format to triplets/macosko2015z.5.1337.idx.npy                    ]8;id=745809;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=98271;file:///home/james/dev/drnb/src/drnb/io/__init__.py#230\230]8;;\

           INFO     Writing numpy format to triplets/macosko2015z.5.1337.l2.npy                     ]8;id=356068;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=64222;file:///home/james/dev/drnb/src/drnb/io/__init__.py#230\230]8;;\

           INFO     Writing pipeline result for macosko2015z                                          ]8;id=640884;file:///home/james/dev/drnb/src/drnb/dataset.py\dataset.py]8;;\:]8;id=738408;file:///home/james/dev/drnb/src/drnb/dataset.py#79\79]8;;\

time: 2min 10s (started: 2022-09-11 21:36:22 -07:00)


### Standardize and PCA=50

To be as close as possible to the procedure in the [openTSNE notebook](https://github.com/pavlin-policar/openTSNE/blob/30b07f1b60129fc74bb937eb4632c6d424f49c4d/examples/prepare_macosko_2015.ipynb), after standardizing PCA is used to reducing the initial dimensionality down to 50 dimensions. Here is that as a pipeline: 

In [39]:
from drnb.dataset import create_data_pipeline

standardize_pca50_pipe = create_data_pipeline(
    convert=dict(dtype="float32", layout="c"),
    scale="z",
    reduce=50,
    data_export=["csv", "npy"],
    target_export=["csv", "pkl"],
    neighbors=dict(
        n_neighbors=[15, 50, 150],
        method="exact",
        metric=["euclidean"],
        file_types=["csv", "npy"],
    ),
    triplets=dict(
        n_triplets_per_point=5,
        seed=1337,
        file_types=["csv", "npy"],
    ),
    verbose=True,
)

[21:39:34] INFO     Requesting one extra neighbor to account for self-neighbor                       ]8;id=820757;file:///home/james/dev/drnb/src/drnb/dataset.py\dataset.py]8;;\:]8;id=764511;file:///home/james/dev/drnb/src/drnb/dataset.py#310\310]8;;\

time: 8.6 ms (started: 2022-09-11 21:39:34 -07:00)


In [40]:
data_result = standardize_pca50_pipe.run(
    "macosko2015z-pca50",
    data=x,
    target=target,
    target_palette=target_palette,
    verbose=True,
)

[21:39:35] INFO     initial data shape: (44808, 3000) target shape: (44808, 2)                        ]8;id=324900;file:///home/james/dev/drnb/src/drnb/dataset.py\dataset.py]8;;\:]8;id=95788;file:///home/james/dev/drnb/src/drnb/dataset.py#45\45]8;;\

           INFO     Removing rows with NAs                                                            ]8;id=496147;file:///home/james/dev/drnb/src/drnb/dataset.py\dataset.py]8;;\:]8;id=567086;file:///home/james/dev/drnb/src/drnb/dataset.py#85\85]8;;\

[21:39:37] INFO     data shape after filtering NAs: (44808, 3000)                                     ]8;id=582221;file:///home/james/dev/drnb/src/drnb/dataset.py\dataset.py]8;;\:]8;id=927326;file:///home/james/dev/drnb/src/drnb/dataset.py#94\94]8;;\

           INFO     Keeping all columns                                                            ]8;id=699963;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=279175;file:///home/james/dev/drnb/src/drnb/preprocess.py#62\62]8;;\

           INFO     data shape after filtering columns: (44808, 3000)                                 ]8;id=910240;file:///home/james/dev/drnb/src/drnb/dataset.py\dataset.py]8;;\:]8;id=601832;file:///home/james/dev/drnb/src/drnb/dataset.py#99\99]8;;\

...

[21:40:23] INFO     Writing csv format to triplets/macosko2015z-pca50.5.1337.l2.csv                 ]8;id=884597;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=941142;file:///home/james/dev/drnb/src/drnb/io/__init__.py#207\207]8;;\

           INFO     Writing numpy format to triplets/macosko2015z-pca50.5.1337.idx.npy              ]8;id=210533;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=824249;file:///home/james/dev/drnb/src/drnb/io/__init__.py#230\230]8;;\

           INFO     Writing numpy format to triplets/macosko2015z-pca50.5.1337.l2.npy               ]8;id=313127;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=641733;file:///home/james/dev/drnb/src/drnb/io/__init__.py#230\230]8;;\

           INFO     Writing pipeline result for macosko2015z-pca50                                    ]8;id=495578;file:///home/james/dev/drnb/src/drnb/dataset.py\dataset.py]8;;\:]8;id=644411;file:///home/james/dev/drnb/src/drnb/dataset.py#79\79]8;;\

time: 48.9 s (started: 2022-09-11 21:39:35 -07:00)
